In [1]:
#!/usr/bin/env python3
import re
from tqdm import tqdm


Tsb_mdfs = [
    "output/simulate/TKSM/C1_chr21/Tsb.mdf",
    "output/simulate/TKSM/C2_chr21/Tsb.mdf",
    "output/simulate/TKSM/C3_chr21/Tsb.mdf",
    "output/simulate/TKSM/C4_chr21/Tsb.mdf",
    "output/simulate/TKSM/C5_chr21/Tsb.mdf",
]


fastq = (
    "output/simulate/TKSM/S1_chr21/Mrg.plA.Tag.SCB.Tag.PCR.Flp.Tag.Trc.Shf.Seq.fastq"
)
mdfs = [
    "output/simulate/TKSM/C1_chr21/Tsb.mdf",
    "output/simulate/TKSM/C2_chr21/Tsb.mdf",
    "output/simulate/TKSM/C3_chr21/Tsb.mdf",
    "output/simulate/TKSM/C4_chr21/Tsb.mdf",
    "output/simulate/TKSM/C5_chr21/Tsb.mdf",
]
fin_mdf = "output/simulate/TKSM/S1_chr21/Mrg.plA.Tag.SCB.Tag.PCR.Flp.Tag.Trc.Shf.mdf"
output = "DELETE.truth.tsv"

In [2]:
tmid_to_tid = dict()
tmid_to_cb = dict()
cb_to_cell_lines = {".": set()}
for mdf in mdfs:
    for line in tqdm(open(mdf), desc=f"Reading {mdf}"):
        if line[0] != "+":
            continue
        line = line.rstrip("\n").split("\t")
        tmid = line[0][1:]
        cell_line = tmid.split(":")[0]
        cb = "."
        tid = ""
        for comment in line[2].split(";"):
            if comment.startswith("CB"):
                comment = comment.split("=")
                if len(comment) == 2:
                    cb = comment[1]
            elif comment.startswith("tid"):
                tid = comment.split("=")[1]
        tmid_to_tid[tmid] = tid
        tmid_to_cb[tmid] = cb
        if cb not in cb_to_cell_lines:
            cb_to_cell_lines[cb] = set()
        cb_to_cell_lines[cb].add(cell_line)

Reading output/simulate/TKSM/C1_chr21/Tsb.mdf: 35403it [00:00, 661515.05it/s]
Reading output/simulate/TKSM/C2_chr21/Tsb.mdf: 35592it [00:00, 812551.94it/s]
Reading output/simulate/TKSM/C3_chr21/Tsb.mdf: 36022it [00:00, 1077240.00it/s]
Reading output/simulate/TKSM/C4_chr21/Tsb.mdf: 49541it [00:00, 1293988.79it/s]
Reading output/simulate/TKSM/C5_chr21/Tsb.mdf: 23783it [00:00, 532402.87it/s]


In [3]:
fmid_to_cb = dict()
fmid_to_tmid = dict()
for line in tqdm(open(fin_mdf), desc=f"Reading {fin_mdf}"):
    line = line.rstrip("\n").split("\t")
    if line[0][0] == "+":
        fmid = line[0][1:]
        tmid = re.split(r"[_.]", fmid)[0]
        tcb = tmid_to_cb[tmid]
        fmid_to_cb[fmid] = "."
        fmid_to_tmid[fmid] = tmid
        continue
    contig, start, end, _, _ = line
    start = int(start)
    end = int(end)
    if line[0] == tcb and (len(tcb) - (end - start) <= 2):
        fmid_to_cb[fmid] = tcb  


Reading output/simulate/TKSM/S1_chr21/Mrg.plA.Tag.SCB.Tag.PCR.Flp.Tag.Trc.Shf.mdf: 1387751it [00:02, 490066.12it/s]


In [4]:
rid_to_fmid = dict()
for idx, line in tqdm(enumerate(open(fastq)), desc=f"Reading {fastq}"):
    if idx % 4 != 0:
        continue
    rid = line.strip().split()[0][1:]
    comments = line.strip().split()[1:]
    for comment in comments:
        if comment.startswith("molecule_id="):
            fmid = comment.split("=")[1]
            assert fmid in fmid_to_tmid and fmid in fmid_to_cb
            rid_to_fmid[rid] = fmid
    assert rid in rid_to_fmid

Reading output/simulate/TKSM/S1_chr21/Mrg.plA.Tag.SCB.Tag.PCR.Flp.Tag.Trc.Shf.Seq.fastq: 957368it [00:01, 578589.79it/s]


In [5]:
outfile = open(output, "w+")
print(
    "\t".join(
        [
            "read_id",
            "transcript_id",
            "cell_barcode",
            "cell_lines",
        ]
    ),
    file=outfile,
)
for rid, fmid in tqdm(rid_to_fmid.items(), desc=f"Writing {output}"):
    tmid = fmid_to_tmid[fmid]
    cb = fmid_to_cb[fmid]
    record = [
        rid,
        tmid_to_tid[tmid],
        cb,
        ",".join(sorted(cb_to_cell_lines[cb])),
    ]
    print("\t".join(record), file=outfile)
outfile.close()

Writing DELETE.truth.tsv:   0%|          | 0/239342 [00:00<?, ?it/s]

Writing DELETE.truth.tsv: 100%|██████████| 239342/239342 [00:00<00:00, 301411.80it/s]


In [7]:
for cb, cell_lines in cb_to_cell_lines.items():
    print(cb, ",".join(sorted(cell_lines)),)

. 
CTCTCAGGTTGTACGT C1
CCCTGATGTTAACTCG C1
GAGACCCAGTCCTACA C1
AAGCATCGTGCTACTA C1
GTATTTCAGAGCCATG C1
TCCTGCACAGGCCTGT C1
ACGTAACAGTTTCTTC C1
TATACCTAGCTGCTAC C1
GACATCACACAACCAT C1
ACTTATCTCCATCCTA C1
AATGCCATCAAGTATC C1
AGGACTTGTTGTTGCA C1
CCGTAGGTCGGACGTC C1
TTTCACAAGAGTCCGA C1
GTCTACCGTTGGACCC C1
GAGCCTGCAATCTTCT C1
TTACCATGTCCACGCA C1
GAAACCTAGGCCTCTA C1
TCATACTTCCTTCACG C1
TACTGCCCACCTCCAT C1
GTTCGCTAGCGCTGAA C1
AGTTCGAAGTCCCAAC C1
GGGACAAAGTGACCCT C1
GCCAGCAAGGCCTAAG C1
CTGAATGCAATGAAAC C1
AGGTTGTTCAGCGTCG C1
CTCATCGCATGATGTC C1
ATGAAAGGTCGCTCGA C1
ATGTCCCCAGGAGGAG C1
GCCAGTGTCCAAATCA C1
GCTTGGGCATATCTGG C1
CACGTTCAGCAAACCA C1
TCTGGCTGTCACCACG C1
TACCTGCTCTATGGCG C1
TTCCAATCAGCATTTG C1
CCAAGCGTCCTACGGG C1
ACTGTGACAGCTATTG C1
TAAGCACTCCAGTACA C1
TCTACATAGCGATTAG C1
TAGGAGGGTTCAGTGT C1
TCGGGCAGTTGACAGG C1
TACCGAAGTCAACACT C1
TTTGACTGTCGACTTA C1
GAGAGGTCAGGCTAGA C1
TGGGTTAAGCAGCCTC C1
CTTACCGAGTGAGTGC C1
CTAGGTAAGACTGATG C1
TTCTAACTCAGTGACC C1
AGGAAATTCCAGCAGT C1
TCAAGTGAGGTGATCG 